In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import flwr as fl
import tensorflow as tf
from collections import OrderedDict, defaultdict
from typing import Dict, List, Optional, Tuple
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch
from flwr.common.logger import log
from logging import INFO, DEBUG
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import TensorDataset, DataLoader, Subset
import torch
import numpy as np

import os

os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [ ]:
DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}"
)

In [ ]:
df_1 = pd.read_csv('features_data/features_cpsc_2018_extra.csv')
df_2 = pd.read_csv('features_data/features_cpsc_2018.csv')
df_3 = pd.read_csv('features_data/features_georgia.csv')
df_4 = pd.read_csv('features_data/features_incart.csv')
df_5 = pd.read_csv('features_data/features_ptb.csv') 
df_6 = pd.read_csv('features_data/features_ptb-xl.csv')

In [ ]:
# merge all dfs
df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6], ignore_index=True)
df.shape

In [ ]:
feature_i_df = pd.read_csv(r"top_120_features.csv")
feature_i_df.head()

In [ ]:
feature_names = list(feature_i_df["Feature Id"])
feature_names[:50]

In [ ]:
df_copy = df.dropna()

In [ ]:
df_copy = df_copy.groupby('label').filter(lambda x: len(x) >= 1100)

In [ ]:
# remove "TAb" and "IAVB" from df_copy
df_copy = df_copy[df_copy['label'] != "TAb"]
df_copy = df_copy[df_copy['label'] != "IAVB"]

df_copy['label'].value_counts()

In [ ]:
# split data into train val and test sets stratified by target

#X = df_copy.drop(columns=['label',"label_abr","merged_labels","id","id.1"])
X = df_copy.drop(columns=['label',"id"])
X = X[feature_names[:150]]
#scale the X




y = df_copy['label']
#convert y into integers
le = LabelEncoder()
y = le.fit_transform(y)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42, stratify=y_train)



In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader, Subset
import torch
import numpy as np

def normalize(X, scaler=None):
    if not scaler:
        scaler = StandardScaler()
        scaler.fit(X)
    return scaler.transform(X), scaler

def create_data_loaders(X_train, y_train, X_val, y_val, X_test, y_test, n_clients):
    # Create the StratifiedKFold object
    skf = StratifiedKFold(n_splits=n_clients)

    # Create the data loaders
    train_loaders = []
    val_loaders = []

    for _, indices in skf.split(X_train, y_train):
        X_subset = X_train.iloc[indices]
        y_subset = y_train[indices]
        X_subset, scaler = normalize(X_subset)
        subset = TensorDataset(torch.from_numpy(X_subset).float(), torch.from_numpy(y_subset).long())
        loader = DataLoader(subset, batch_size=32)
        train_loaders.append(loader)

    for _, indices in skf.split(X_val, y_val):
        X_subset = X_val.iloc[indices]
        y_subset = y_val[indices]
        X_subset, _ = normalize(X_subset, scaler)
        subset = TensorDataset(torch.from_numpy(X_subset).float(), torch.from_numpy(y_subset).long())
        loader = DataLoader(subset, batch_size=32)
        val_loaders.append(loader)

    # Normalize the test data based on the training data and create a DataLoader
    X_test, _ = normalize(X_test, scaler)
    test_dataset = TensorDataset(torch.from_numpy(X_test).float(), torch.from_numpy(y_test).long())
    test_loader = DataLoader(test_dataset, batch_size=32)

    return train_loaders, val_loaders, test_loader

n_clients = 8
train_loaders, val_loaders,test_loader = create_data_loaders(X_train, y_train, X_val, y_val,X_test,y_test, n_clients)

In [ ]:
# Initialize a figure
fig, axs = plt.subplots(4,2, figsize=(14, 8))
axs = axs.flatten()
# Loop over each train loader
for i, train_loader in enumerate(train_loaders):
    # Get labels from train loader
    labels = [label for _, label in train_loader]
    labels = np.concatenate(labels)  # Concatenate list of tensors into a single numpy array

   # Convert labels to pandas Series
    labels_series = pd.Series(labels)

    # Count occurrences of each label
    label_counts = labels_series.value_counts().sort_index()

    # Plot horizontal bar chart of label counts
    label_counts.plot(kind='barh', ax=axs[i])
    axs[i].set_title(f'Client {i+1}')


plt.tight_layout()
plt.show()

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(150, 128)
        self.fc2 = nn.Linear(128, 500)
        self.fc4 = nn.Linear(500,5)

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        #x = nn.functional.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

def train(net, trainloader, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()

    with torch.no_grad():
        true_labels = []
        pred_labels = []
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(predicted.cpu().numpy())

    loss /= len(testloader.dataset)
    accuracy = correct/total
    # Calculate F1 score
    f1 = f1_score(true_labels, pred_labels, average='macro')
    precision = precision_score(true_labels, pred_labels, average='macro')
    recall = recall_score(true_labels, pred_labels, average='macro')
    return loss, accuracy,f1, precision, recall, true_labels, pred_labels

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader

    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)

    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader, epochs=5)
        return get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy,f1, precision, recall,true_labels, pred_labels = test(self.net, self.valloader)
        # save_path = r"Non-IID Dist Scaling\8 clients\classification_report_client_"+str(self.cid)+"_8_clients.csv"
        # # plt.savefig(save_path)
        # # plt.show()
        # #save the plot of confusion matrix
        # #get classification report  and save it
        # #get the classification report
        # from sklearn.metrics import classification_report
        # report = classification_report(true_labels, pred_labels,target_names=le.classes_,output_dict=True)
        # report_df = pd.DataFrame(report).transpose()
        # report_df.to_csv(save_path)
        log(INFO, f"Client side accuracy: {float(accuracy)}")
        return float(loss), len(self.valloader),{"accuracy": float(accuracy),
                                                 "f1": float(f1),
                                                 "precision":float(precision),
                                                 "recall":float(recall)}
    


def client_fn(cid) -> FlowerClient:
    net = Net().to(DEVICE)
    trainloader = train_loaders[int(cid)]
    valloader = val_loaders[int(cid)]
    return FlowerClient(cid, net, trainloader, valloader)

In [ ]:
# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = test_loader
    set_parameters(net, parameters)  # Update model with the latest parameters
    loss, accuracy,f1, precision, recall, true_labels, pred_labels = test(net, valloader)
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='g', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title(f'Confusion Matrix for Server Evaluation')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    print(f"Server-side evaluation loss {loss} / accuracy {accuracy}")
    return loss, {"accuracy": accuracy,
                    "f1":float(f1),
                    "precision":float(precision),
                    "recall":float(recall)}





In [ ]:
def average_metrics(metrics):
    """Aggregate metrics from multiple clients by calculating mean averages.

    Parameters:
    - metrics (list): A list containing tuples, where each tuple represents metrics for a client.
                    Each tuple is structured as (num_examples, metric), where:
                    - num_examples (int): The number of examples used to compute the metrics.
                    - metric (dict): A dictionary containing custom metrics provided as `output_dict`
                                    in the `evaluate` method from `client.py`.

    Returns:
    A dictionary with the aggregated metrics, calculating mean averages. The keys of the
    dictionary represent different metrics, including:
    - 'accuracy': Mean accuracy calculated by TensorFlow.
    - 'acc': Mean accuracy from scikit-learn.
    - 'rec': Mean recall from scikit-learn.
    - 'prec': Mean precision from scikit-learn.
    - 'f1': Mean F1 score from scikit-learn.

    Note: If a weighted average is required, the `num_examples` parameter can be leveraged.

    Example:
        Example `metrics` list for two clients after the last round:
        [(10000, {'prec': 0.108, 'acc': 0.108, 'f1': 0.108, 'accuracy': 0.1080000028014183, 'rec': 0.108}),
        (10000, {'f1': 0.108, 'rec': 0.108, 'accuracy': 0.1080000028014183, 'prec': 0.108, 'acc': 0.108})]
    """

    # Here num_examples are not taken into account by using _
    accuracies_tf = np.mean([metric["accuracy"] for _, metric in metrics])
    # accuracies = np.mean([metric["acc"] for _, metric in metrics])
    recalls = np.mean([metric["recall"] for _, metric in metrics])
    precisions = np.mean([metric["precision"] for _, metric in metrics])
    f1s = np.mean([metric["f1"] for _, metric in metrics])

    return {
        "accuracy": accuracies_tf,
        # "acc": accuracies,
        "rec": recalls,
        "prec": precisions,
         "f1": f1s,
    }

In [ ]:
# Create an instance of the model and get the parameters
params = get_parameters(Net())

# Pass parameters to the Strategy for server-side parameter initialization
strategy = fl.server.strategy.FedAvg(
    fraction_fit=0.3,
    fraction_evaluate=0.3,
    min_fit_clients=2,
    min_evaluate_clients=2,
    min_available_clients=n_clients,
    initial_parameters=fl.common.ndarrays_to_parameters(params),
    evaluate_fn = evaluate,
    evaluate_metrics_aggregation_fn = average_metrics
)

# strategy = AggregateCustomMetricStrategy(
#     fraction_fit=0.7,
#     fraction_evaluate=0.3,
#     min_fit_clients=2,
#     min_evaluate_clients=2,
#     min_available_clients=n_clients,
#     initial_parameters=fl.common.ndarrays_to_parameters(params),
#     evaluate_fn = evaluate,
# )


# Specify client resources if you need GPU (defaults to 1 CPU and 0 GPU)
client_resources = None
if DEVICE.type == "cuda":
    client_resources = {"num_gpus": 1}

# Start simulation
history_sim = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=n_clients,
    config=fl.server.ServerConfig(num_rounds=15),  # Just three rounds
    strategy=strategy,
    client_resources=client_resources,
    ray_init_args={"include_dashboard": True}
)

In [ ]:
#save all metrics from history_sim as df csv 

history_distributed = pd.DataFrame(history_sim.metrics_distributed)
history_centralized = pd.DataFrame(history_sim.metrics_centralized)

